In [1]:
#ライブラリインポート
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import time
from tqdm.notebook import tqdm
import pandas as pd

1:札幌　2:函館　3:福島　4:新潟　5:東京　6:中山　7:中京　8:京都　9:阪神　10:小倉

In [2]:
#レースIDのリスト作成
#URL構造　"https://db.netkeiba.com/race/"　+　race_id
#race_id構造　年,開催場所,回数,何日目か,レース

race_id_list = []
for year in range(2008,2022):
    #(2008,2021)
    for place in range(1, 11):
        #(1, 11)
        for kai in range(1, 7):
            #(1,7)
            for day in range(1, 13):
                #(1,13)
                for r in range(1, 13):
                    #(1,13)
                    race_id = str(year).zfill(4) + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                    race_id_list.append(race_id)



In [3]:
#レース情報についてスクレイピングする関数定義

def scrape_race_info(race_id_list):
    race_infos = {}
    for race_id in tqdm(race_id_list):
        try:
            #レース結果ページから、レースの情報について収集し、整形する
            html = urlopen('https://db.netkeiba.com/race/' + race_id)
            bs = BeautifulSoup(html.read(),'html.parser')

            #レース情報についてスクレイピング
            #[レース名、距離、天気、馬場、馬場状態、日付]
            texts = (
                        bs.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                        + bs.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                        )
            race_name = bs.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
            #infoに正規表現を使ってtextsを取得
            info = re.findall(r"\w+", texts)

            #整形
            info_dict = {}
            for text in info:
                info_dict["race_name"] = race_name
                if "m" in text:
                    info_dict["course_len"] = int(re.findall(r"\d+", text)[0])
                if "m" in text:
                    info_dict["rotation"] = str(re.findall(r"[右,左]", text)[0])
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    info_dict["weather"] = text
                if text in ["芝", "ダート"]:
                    info_dict["race_type"] = text
                if text in ["良", "稍重", "重", "不良"]:
                    info_dict["ground_state"] = text
                if "障" in text:
                    info_dict["race_type"] = "障害"
                if "年" in text:
                    info_dict["date"] = text
        
            race_infos[race_id] = info_dict
            #time.sleep(1)
        except IndexError:
            continue
        except AttributeError:
            continue
        except Exception as e:
            print(e)
            break
        except:
            break
    return race_infos

In [4]:
#関数実行
race_infos=scrape_race_info(race_id_list)

  0%|          | 0/120960 [00:00<?, ?it/s]

In [5]:
#DataFrame型に変更
race_infos = pd.DataFrame(race_infos)
race_infos_T = race_infos.T
race_infos_all = race_infos_T.rename_axis('race_id').reset_index()

#df表示
race_infos_all


,race_id,race_name,course_len,rotation,weather,race_type,ground_state,date
0,200801010101,2歳未勝利,1500,右,曇,芝,良,2008年8月16日
1,200801010102,3歳未勝利,1000,右,曇,ダート,稍重,2008年8月16日
2,200801010103,3歳未勝利,1700,右,曇,ダート,稍重,2008年8月16日
3,200801010104,2歳新馬,1500,右,曇,芝,良,2008年8月16日
4,200801010105,3歳未勝利,1700,右,曇,ダート,稍重,2008年8月16日
...,...,...,...,...,...,...,...,...
40402,202110040808,3歳以上1勝クラス,1000,右,曇,ダート,稍重,2021年9月5日
40403,202110040809,筑後川特別,1800,右,小雨,芝,良,2021年9月5日
40404,202110040810,西日本スポーツ杯,1700,右,小雨,ダート,稍重,2021年9月5日
40405,202110040811,第41回小倉2歳ステークス(G3),1200,右,小雨,芝,良,2021年9月5日


In [6]:

#CSV形式で保存
race_infos_all.to_csv("race_infos.csv",encoding="shift_jis")